# Cycistic case study

### A Google Certificate case study for Data Analytics. The marketing analyst team wants to understand how casual riders and annual members use Cyclistic bikes differently in Chicago.

### Summary

- Launch in 2016
- The program has grown to a fleet of 5,524 bicycles
- Geotracked and locked into a network of 692 stations across Chicago
- Marketing strategy was to build general awereness to a broad consumer segment

- Pricing plan is flexible:
    - casual riders: single-ride or full-day passes
    - annual membership

Although the pricing flexibility helps Cyclistic attract more customers, the team believes that __maximizing the number of annual members will be the key to future growth__. Rather than creating a marketing campaign that targets all-new customers, there might be a very good chance to convert casual riders into members. The problem is that __the casual riders__ are already aware of the Cyclistic program and __have already chosen__ Cyclistic __for their mobility needs__.

***

### Describe the business task:

>_Key stakeholders: Lily Moreno (The director of marketing and the manager), Cyclistic marketing analytics team and the  Cyclistic executive team_

The clear goal is to design marketing strategies aimed at converting casual riders into annual members. In order to
do that, however, the marketing analyst team needs to __better understand how annual members and casual riders differ, why
casual riders would buy a membership, and how digital media could affect their marketing tactics__.

There are three questions that will guide the future marketing program:
1. How do annual members and casual riders use Cyclistic bikes differently?
2. Why would casual riders buy Cyclistic annual memberships?
3. How can Cyclistic use digital media to influence casual riders to become members?

My scope would be to analyze the first question with the previous 12 months of __[Cyclistic historical bike trip data](https://divvy-tripdata.s3.amazonaws.com/index.html)__ to identify trends.


### **** Disclaimer ****
> Cyclisist is a fictional company. For the sole purpose of this case study the historical datasets are appropriate and can be used to answer the business questions. Nevertheless, the owner of the datasets is Motivate International Inc. under __[this licence.](https://ride.divvybikes.com/data-license-agreement)__ Motivate International Inc. disclaims any warranty that data will meet your needs or will be or continue to be available, complete, accurate, timely, secure or error free.


### The data

These datasets for the sole purpose of this case study is ROCCC:
- Reliable, Original and Cited: complete time series, which are accurate for the task and is non-bias. We will treat them as second-party datasets from a reliable organization.
- Current and Comprehensive: we consider them as regularly refreshed datasets, which are appropriate and will enable us to answer the business questions. 

Data has been downloaded and stored locally and uploaded into Google Drive as a back-up.

I wasn't able to choose the quarterly files, because the latest was back in 2020, so I downloaded the monthly versions in csv from March of 2022 until February 2023.

These files are structured, orgaized in 13 columns without indicies including quantitative and qualitative data.

I will use MySQL connector to export the database from data table into a csv file because MySQL would be much slower to do it.

In [ ]:
import mysql.connector
import csv
import sys

db = mysql.connector.connect(
  host='localhost',
  user='root',
  password='*********',
  database='cyclistic'
)

cursor = db.cursor()

query = 'SELECT * FROM data'

csv_path = 'C:/Users/Betti/Desktop/case study/historical data/alldata.csv'

try:
    cursor.execute(query)
    rows = cursor.fetchall()
finally:
    db.close()

# Continue only if there are rows returned.
if rows:
    # New empty list called 'result'. This will be written to a file.
    result = list()

    # The row name is the first entry for each entity in the description tuple.
    column_names = list()
    for i in cursor.description:
        column_names.append(i[0])

    result.append(column_names)
    for row in rows:
        result.append(row)

    # Write result to file.
    with open(csv_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in result:
            csvwriter.writerow(row)
else:
    sys.exit("No rows found for query: {}".format(query))

In [1]:
import pandas as pd
df = pd.read_csv('alldata.csv')

In [2]:
df.shape

(5829084, 13)

In [3]:
df.index

RangeIndex(start=0, stop=5829084, step=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5829084 entries, 0 to 5829083
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 578.1+ MB


In [5]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,47EC0A7F82E65D52,classic_bike,2022-03-21 13:45:01,2022-03-21 13:51:18,Wabash Ave & Wacker Pl,TA1307000131,Kingsbury St & Kinzie St,KA1503000043,41.8869,-87.6260,41.8892,-87.6385,member
1,8494861979B0F477,electric_bike,2022-03-16 09:37:16,2022-03-16 09:43:34,Michigan Ave & Oak St,13042,Orleans St & Chestnut St (NEXT Apts),620,41.9010,-87.6237,41.8982,-87.6375,member
2,EFE527AF80B66109,classic_bike,2022-03-23 19:52:02,2022-03-23 19:54:48,Broadway & Berwyn Ave,13109,Broadway & Ridge Ave,15578,41.9784,-87.6598,41.9840,-87.6603,member
3,9F446FD9DEE3F389,classic_bike,2022-03-01 19:12:26,2022-03-01 19:22:14,Wabash Ave & Wacker Pl,TA1307000131,Franklin St & Jackson Blvd,TA1305000025,41.8869,-87.6260,41.8777,-87.6353,member
4,431128AD9AFFEDC0,classic_bike,2022-03-21 18:37:01,2022-03-21 19:19:11,DuSable Lake Shore Dr & North Blvd,LF-005,Loomis St & Jackson Blvd,13206,41.9117,-87.6268,41.8779,-87.6620,member


In [7]:
# Let's check if we have interesting statistical data on the numeric data in our dataset
df.describe(include=object)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,member_casual
count,5829084,5829084,5829084,5829084,4978666,4978534,4920046,4919905,5829084
unique,5829084,3,4891324,4904887,1694,1314,1717,1318,4
top,47EC0A7F82E65D52,electric_bike,2022-05-30 13:05:15,2022-08-22 12:47:49,Streeter Dr & Grand Ave,13022,Streeter Dr & Grand Ave,13022,member
freq,1,2983084,9,20,75992,75992,76152,76152,3059323


In [2]:
df['started_at'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d %H:%M:%S')
df['ended_at'] = pd.to_datetime(df['ended_at'], format='%Y-%m-%d %H:%M:%S')

In [25]:
df['ride_id2'] = df.to_string(df['ride_id'])

MemoryError: Unable to allocate 44.5 MiB for an array with shape (5829084,) and data type int64

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5829084 entries, 0 to 5829083
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 578.1+ MB


In [23]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,47EC0A7F82E65D52,classic_bike,2022-03-21 13:45:01,2022-03-21 13:51:18,Wabash Ave & Wacker Pl,TA1307000131,Kingsbury St & Kinzie St,KA1503000043,41.8869,-87.6260,41.8892,-87.6385,member
1,8494861979B0F477,electric_bike,2022-03-16 09:37:16,2022-03-16 09:43:34,Michigan Ave & Oak St,13042,Orleans St & Chestnut St (NEXT Apts),620,41.9010,-87.6237,41.8982,-87.6375,member
2,EFE527AF80B66109,classic_bike,2022-03-23 19:52:02,2022-03-23 19:54:48,Broadway & Berwyn Ave,13109,Broadway & Ridge Ave,15578,41.9784,-87.6598,41.9840,-87.6603,member
3,9F446FD9DEE3F389,classic_bike,2022-03-01 19:12:26,2022-03-01 19:22:14,Wabash Ave & Wacker Pl,TA1307000131,Franklin St & Jackson Blvd,TA1305000025,41.8869,-87.6260,41.8777,-87.6353,member
4,431128AD9AFFEDC0,classic_bike,2022-03-21 18:37:01,2022-03-21 19:19:11,DuSable Lake Shore Dr & North Blvd,LF-005,Loomis St & Jackson Blvd,13206,41.9117,-87.6268,41.8779,-87.6620,member
